# Optimized Algorithm

In [1]:
import requests
import pandas as pd
import os
import datetime
import time 
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import glob
import pandas as pd
import urllib, json
import re
from itertools import permutations
from pulp import *

def algorithm():
    driver = webdriver.Chrome(executable_path=r"C:\\Users\\Matt\\Downloads\\chromedriver.exe")

    url = "https://www.fanduel.com/contests/mlb/96"
    user = "Matthewdorris7@gmail.com"
    password_name = "Gonzaga@1"

    driver.get(url)
    wait = WebDriverWait(driver,100)

    username = wait.until(EC.presence_of_element_located((By.XPATH,"""//*[@id="forms.login.email"]""")))
    username.send_keys(user)

    password = wait.until(EC.presence_of_element_located((By.XPATH,"""//*[@id="forms.login.password"]""")))
    password.send_keys(password_name)

    logInBtn = wait.until(EC.element_to_be_clickable((By.XPATH,"""//*[@id="forms.login.submit"]""")))
    logInBtn.click()

    sortBtn = wait.until(EC.element_to_be_clickable((By.XPATH,"""//*[@id="root"]/div[1]/main/div/main/div[1]/div[4]/button/span[2]""")))
    sortBtn.click()

    feeBtn = wait.until(EC.element_to_be_clickable((By.XPATH,"""//*[@id="root"]/div[1]/main/div/main/div[1]/div[4]/div/section/div/div[2]/span""")))
    feeBtn.click()

    contestBtn = wait.until(EC.element_to_be_clickable((By.XPATH,"""//*[@id="contestList"]/div/div[1]/div/div/div[1]/div/a""")))
    contestBtn.click()

    playersBtn = wait.until(EC.element_to_be_clickable((By.XPATH,"""//*[@id="global-view"]/div/section/div[2]/roster-editor/div/div[1]/section/div[1]/div/a[2]""")))
    playersBtn.click()

    list_of_files = glob.glob(r'C:\Users\Matt\Downloads\*.csv') # * means all if need specific format then *.csv
    fileName = max(list_of_files, key=os.path.getctime)

    #fileName = r'C:\Users\Matt\Downloads\FanDuel-MLB-2019-08-17-37743-players-list (1).csv'

    data = pd.read_csv(fileName)
    data = data[data['Injury Indicator'].isnull()]
    indexNames = data[data['Probable Pitcher']  != 'Yes'].index & data[data['Position']  == 'P'].index
    data.drop(indexNames , inplace=True)
    data = data[(data['Batting Order']  != 0)]

    current = data[['Position', 'Nickname', 'Salary' , 'FPPG']]
    #current = current.replace('C','C/1B')
    #current = current.replace('1B','C/1B')
    # Add utility position
    #util = current[current.Position.isin(["C/1B","2B","3B", "SS", "OF"])].copy()
    #util.Position = "UTIL"
    #current = pd.concat([current, util])

    availables = current[["Position", "Nickname", "Salary","FPPG"]].groupby(["Position", "Nickname", "Salary", "FPPG"]).agg("count")
    availables = availables.reset_index()
    salaries = {}
    points = {}
    for pos in availables.Position.unique():
        available_pos = availables[availables.Position == pos]
        salary = list(available_pos[["Nickname","Salary"]].set_index("Nickname").to_dict().values())[0]
        point = list(available_pos[["Nickname","FPPG"]].set_index("Nickname").to_dict().values())[0]
        salaries[pos] = salary
        points[pos] = point
    pos_num_available = {
        "P": 1,
        "C": 1,
        "1B": 1,
        "2B": 1,
        "3B": 1, 
        "SS": 1,
        "OF": 3, 
        #"UTIL": 1

    }
    SALARY_CAP = 35000
    _vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}
    prob = LpProblem("Fantasy", LpMaximize)
    rewards = []
    costs = []
    position_constraints = []
    # Setting up the reward
    for k, v in _vars.items():
        costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
        rewards += lpSum([points[k][i] * _vars[k][i] for i in v])
        prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]

    prob += lpSum(rewards)
    prob += lpSum(costs) <= SALARY_CAP
    prob.solve()

    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))

In [75]:
%%time
algorithm()

Variables:

1B_Matt_Olson = 1.0
2B_Cavan_Biggio = 1.0
3B_Rafael_Devers = 1.0
C_Tom_Murphy = 1.0
OF_Hunter_Renfroe = 1.0
OF_Mallex_Smith = 1.0
OF_Mookie_Betts = 1.0
P_Joey_Lucchesi = 1.0
SS_Bo_Bichette = 1.0
---------------------------------------

Constraints:
3300*1.0 + 3200*1.0 + 4400*1.0 + 3000*1.0 + 2700*1.0 + 2800*1.0 + 4200*1.0 + 7400*1.0 + 4000*1.0 = 35000.0
---------------------------------------

Score:
11.366316303453946*1.0 + 9.90274006046661*1.0 + 14.084251584030511*1.0 + 10.631481029369212*1.0 + 9.887825874660326*1.0 + 10.053773124262973*1.0 + 13.419379655704944*1.0 + 28.33333333333333*1.0 + 14.584000244140626*1.0 = 122.26310120942247
Wall time: 25.5 s


# Stacked Algorithm

In [77]:
import requests
import pandas as pd
import os
import datetime
import time 
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import glob
import pandas as pd
import urllib, json
import re
from itertools import permutations
from pulp import *

driver = webdriver.Chrome(executable_path=r"C:\\Users\\Matt\\Downloads\\chromedriver.exe")

url = "https://www.fanduel.com/contests/mlb/96"
user = "Matthewdorris7@gmail.com"
password_name = "Gonzaga@1"

driver.get(url)
wait = WebDriverWait(driver,100)

username = wait.until(EC.presence_of_element_located((By.XPATH,"""//*[@id="forms.login.email"]""")))
username.send_keys(user)

password = wait.until(EC.presence_of_element_located((By.XPATH,"""//*[@id="forms.login.password"]""")))
password.send_keys(password_name)

logInBtn = wait.until(EC.element_to_be_clickable((By.XPATH,"""//*[@id="forms.login.submit"]""")))
logInBtn.click()

sortBtn = wait.until(EC.element_to_be_clickable((By.XPATH,"""//*[@id="root"]/div[1]/main/div/main/div[1]/div[4]/button/span[2]""")))
sortBtn.click()

feeBtn = wait.until(EC.element_to_be_clickable((By.XPATH,"""//*[@id="root"]/div[1]/main/div/main/div[1]/div[4]/div/section/div/div[2]/span""")))
feeBtn.click()

contestBtn = wait.until(EC.element_to_be_clickable((By.XPATH,"""//*[@id="contestList"]/div/div[1]/div/div/div[1]/div/a""")))
contestBtn.click()

playersBtn = wait.until(EC.element_to_be_clickable((By.XPATH,"""//*[@id="global-view"]/div/section/div[2]/roster-editor/div/div[1]/section/div[1]/div/a[2]""")))
playersBtn.click()

list_of_files = glob.glob(r'C:\Users\Matt\Downloads\*.csv') # * means all if need specific format then *.csv
fileName = max(list_of_files, key=os.path.getctime)



In [78]:
f = open('lineups.csv','w')

def stack_teams(team_1, team_2):

    data = pd.read_csv(fileName)
    data = data[data['Injury Indicator'].isnull()]
    indexNames = data[data['Probable Pitcher']  != 'Yes'].index & data[data['Position']  == 'P'].index
    data.drop(indexNames , inplace=True)
    data = data[(data['Batting Order']  != 0)]
    
    pitchers = data[data['Probable Pitcher']  == 'Yes']
    team1 = data[data['Team']  == team_1] 
    team2 = data[data['Team']  == team_2] 
    data= pd.concat([team1, team2])
    data= pd.concat([pitchers, data])
    data.drop(data[data['Position'] == 'P'].index & data[data['Team'] == team_1].index, inplace=True)
    data.drop(data[data['Position'] == 'P'].index & data[data['Team'] == team_2].index, inplace=True)

    current = data[['Position', 'Nickname', 'Salary' , 'FPPG']]

    #current = current.replace('C','C/1B')
    #current = current.replace('1B','C/1B')
    # Add utility position
    #util = current[current.Position.isin(["C/1B","2B","3B", "SS", "OF"])].copy()
    #util.Position = "UTIL"
    #current = pd.concat([current, util])

    availables = current[["Position", "Nickname", "Salary","FPPG"]].groupby(["Position", "Nickname", "Salary", "FPPG"]).agg("count")
    availables = availables.reset_index()
    salaries = {}
    points = {}
    for pos in availables.Position.unique():
        available_pos = availables[availables.Position == pos]
        salary = list(available_pos[["Nickname","Salary"]].set_index("Nickname").to_dict().values())[0]
        point = list(available_pos[["Nickname","FPPG"]].set_index("Nickname").to_dict().values())[0]
        salaries[pos] = salary
        points[pos] = point
    pos_num_available = {
        "P": 1,
        "C": 1,
        "1B": 1,
        "2B": 1,
        "3B": 1, 
        "SS": 1,
        "OF": 3, 
        #"UTIL": 1

    }
    SALARY_CAP = 35000
    _vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}
    prob = LpProblem("Fantasy", LpMaximize)
    rewards = []
    costs = []
    position_constraints = []
    # Setting up the reward
    for k, v in _vars.items():
        costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
        rewards += lpSum([points[k][i] * _vars[k][i] for i in v])
        prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]

    prob += lpSum(rewards)
    prob += lpSum(costs) <= SALARY_CAP
    prob.solve()

    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)

    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
            f.write(str(v.name))
            f.write(",")
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))
    
    f.write(str(eval(score)))
    f.write('\n')


In [79]:
%%time
from itertools import combinations
teams = data.Team.unique()
teams
for team_1,team_2 in combinations(teams, 2):
    stack_teams(team_1, team_2)
    
f.close()

Variables:

1B_Pete_Alonso = 1.0
2B_Jeff_McNeil = 1.0
3B_Todd_Frazier = 1.0
C_Will_Smith = 1.0
OF_Carlos_Gomez = 1.0
OF_Cody_Bellinger = 1.0
OF_Michael_Conforto = 1.0
P_Lance_Lynn = 1.0
SS_Amed_Rosario = 1.0
---------------------------------------

Constraints:
4100*1.0 + 2900*1.0 + 2500*1.0 + 3400*1.0 + 2000*1.0 + 4600*1.0 + 3300*1.0 + 9300*1.0 + 2800*1.0 = 34900.0
---------------------------------------

Score:
12.964122510138356*1.0 + 11.367889684274655*1.0 + 8.436110885054976*1.0 + 13.402941535500917*1.0 + 6.117647058823528*1.0 + 14.644615760216348*1.0 + 11.694261894851437*1.0 + 38.37037037037037*1.0 + 9.692187309265137*1.0 = 126.69014700849571
Variables:

1B_Pete_Alonso = 1.0
2B_Jeff_McNeil = 1.0
3B_Todd_Frazier = 1.0
C_Wilson_Ramos = 1.0
OF_Carlos_Gomez = 1.0
OF_J.D._Davis = 1.0
OF_Michael_Conforto = 1.0
P_Lance_Lynn = 1.0
SS_Amed_Rosario = 1.0
---------------------------------------

Constraints:
4100*1.0 + 2900*1.0 + 2500*1.0 + 2400*1.0 + 2000*1.0 + 2900*1.0 + 3300*1.0 + 9300*1

Variables:

1B_Pete_Alonso = 1.0
2B_Jeff_McNeil = 1.0
3B_Todd_Frazier = 1.0
C_Wilson_Ramos = 1.0
OF_Carlos_Gomez = 1.0
OF_J.D._Davis = 1.0
OF_Michael_Conforto = 1.0
P_Lance_Lynn = 1.0
SS_Amed_Rosario = 1.0
---------------------------------------

Constraints:
4100*1.0 + 2900*1.0 + 2500*1.0 + 2400*1.0 + 2000*1.0 + 2900*1.0 + 3300*1.0 + 9300*1.0 + 2800*1.0 = 32200.0
---------------------------------------

Score:
12.964122510138356*1.0 + 11.367889684274655*1.0 + 8.436110885054976*1.0 + 8.399999895666399*1.0 + 6.117647058823528*1.0 + 8.187825874660328*1.0 + 11.694261894851437*1.0 + 38.37037037037037*1.0 + 9.692187309265137*1.0 = 115.23041548310518
Variables:

1B_Pete_Alonso = 1.0
2B_Jeff_McNeil = 1.0
3B_Todd_Frazier = 1.0
C_Wilson_Ramos = 1.0
OF_Carlos_Gomez = 1.0
OF_J.D._Davis = 1.0
OF_Michael_Conforto = 1.0
P_Lance_Lynn = 1.0
SS_Amed_Rosario = 1.0
---------------------------------------

Constraints:
4100*1.0 + 2900*1.0 + 2500*1.0 + 2400*1.0 + 2000*1.0 + 2900*1.0 + 3300*1.0 + 9300*1.0 +

Variables:

1B_Edwin_Rios = 1.0
2B_Kike_Hernandez = 1.0
3B_Justin_Turner = 1.0
C_Will_Smith = 1.0
OF_Chris_Taylor = 1.0
OF_Cody_Bellinger = 1.0
OF_Joc_Pederson = 1.0
P_Lance_Lynn = 1.0
SS_Corey_Seager = 1.0
---------------------------------------

Constraints:
2000*1.0 + 2900*1.0 + 3500*1.0 + 3400*1.0 + 2800*1.0 + 4600*1.0 + 3300*1.0 + 9300*1.0 + 3200*1.0 = 35000.0
---------------------------------------

Score:
7.270588145536535*1.0 + 8.884111921363901*1.0 + 10.6869918699187*1.0 + 13.402941535500917*1.0 + 8.388775261080994*1.0 + 14.644615760216348*1.0 + 9.167479476308435*1.0 + 38.37037037037037*1.0 + 10.148148148148149*1.0 = 120.96402248844434
Variables:

1B_Garrett_Cooper = 1.0
2B_Kike_Hernandez = 1.0
3B_Justin_Turner = 1.0
C_Will_Smith = 1.0
OF_A.J._Pollock = 1.0
OF_Cody_Bellinger = 1.0
OF_Harold_Ramirez = 1.0
P_Lance_Lynn = 1.0
SS_Corey_Seager = 1.0
---------------------------------------

Constraints:
2400*1.0 + 2900*1.0 + 3500*1.0 + 3400*1.0 + 3500*1.0 + 4600*1.0 + 2200*1.0 + 930

Variables:

2B_Ryan_McMahon = 1.0
3B_Nolan_Arenado = 1.0
C_Dom_Nunez = 1.0
OF_Charlie_Blackmon = 1.0
OF_Ian_Desmond = 1.0
OF_Sam_Hilliard = 1.0
P_Lance_Lynn = 1.0
SS_Trevor_Story = 1.0
---------------------------------------

Constraints:
3500*1.0 + 4700*1.0 + 3100*1.0 + 4400*1.0 + 3200*1.0 + 2500*1.0 + 9300*1.0 + 4300*1.0 = 35000.0
---------------------------------------

Score:
9.896521526834238*1.0 + 12.78091640326813*1.0 + 8.680000305175781*1.0 + 13.40173870584239*1.0 + 8.594915357686705*1.0 + 12.600000381469727*1.0 + 38.37037037037037*1.0 + 13.741666666666667*1.0 = 118.066129717314
Variables:

P_Jacob_deGrom = 1.0
---------------------------------------

Constraints:
11500*1.0 = 11500.0
---------------------------------------

Score:
40.481481481481474*1.0 = 40.481481481481474
Variables:

P_Jacob_deGrom = 1.0
---------------------------------------

Constraints:
11500*1.0 = 11500.0
---------------------------------------

Score:
40.481481481481474*1.0 = 40.481481481481474
Variable

Variables:

P_Jacob_deGrom = 1.0
---------------------------------------

Constraints:
11500*1.0 = 11500.0
---------------------------------------

Score:
40.481481481481474*1.0 = 40.481481481481474
Variables:

1B_Eric_Hosmer = 1.0
2B_Greg_Garcia = 1.0
3B_Manny_Machado = 1.0
C_Dom_Nunez = 1.0
OF_Charlie_Blackmon = 1.0
OF_Hunter_Renfroe = 1.0
OF_Sam_Hilliard = 1.0
P_Lance_Lynn = 1.0
SS_Trevor_Story = 1.0
---------------------------------------

Constraints:
3000*1.0 + 2200*1.0 + 3500*1.0 + 3100*1.0 + 4400*1.0 + 2600*1.0 + 2500*1.0 + 9300*1.0 + 4300*1.0 = 34900.0
---------------------------------------

Score:
10.072519083969466*1.0 + 5.839047386532737*1.0 + 10.6875*1.0 + 8.680000305175781*1.0 + 13.40173870584239*1.0 + 9.601653296100208*1.0 + 12.600000381469727*1.0 + 38.37037037037037*1.0 + 13.741666666666667*1.0 = 122.99449619612733
Variables:

1B_Eric_Hosmer = 1.0
2B_Greg_Garcia = 1.0
3B_Manny_Machado = 1.0
C_Francisco_Mejia = 1.0
OF_Hunter_Renfroe = 1.0
OF_Manuel_Margot = 1.0
OF_Wil_M

Variables:

1B_Kevin_Cron = 1.0
2B_Ryan_McMahon = 1.0
3B_Eduardo_Escobar = 1.0
C_Alex_Avila = 1.0
OF_Adam_Jones = 1.0
OF_Charlie_Blackmon = 1.0
OF_Sam_Hilliard = 1.0
P_Lance_Lynn = 1.0
SS_Trevor_Story = 1.0
---------------------------------------

Constraints:
2000*1.0 + 3500*1.0 + 3700*1.0 + 2300*1.0 + 2800*1.0 + 4400*1.0 + 2500*1.0 + 9300*1.0 + 4300*1.0 = 34800.0
---------------------------------------

Score:
7.12962962962963*1.0 + 9.896521526834238*1.0 + 12.242307692307694*1.0 + 8.491666793823242*1.0 + 8.973949784992122*1.0 + 13.40173870584239*1.0 + 12.600000381469727*1.0 + 38.37037037037037*1.0 + 13.741666666666667*1.0 = 124.84785155193606
Variables:

1B_Danny_Santana = 1.0
2B_Nick_Solak = 1.0
3B_Logan_Forsythe = 1.0
C_Tony_Wolters = 1.0
OF_Charlie_Blackmon = 1.0
OF_Sam_Hilliard = 1.0
OF_Shin_soo_Choo = 1.0
P_Hyun_Jin_Ryu = 1.0
SS_Trevor_Story = 1.0
---------------------------------------

Constraints:
3400*1.0 + 2900*1.0 + 2300*1.0 + 2600*1.0 + 4400*1.0 + 2500*1.0 + 3500*1.0 + 90

Variables:

1B_Danny_Santana = 1.0
2B_Rougned_Odor = 1.0
3B_Logan_Forsythe = 1.0
C_Tim_Federowicz = 1.0
OF_Delino_DeShields = 1.0
OF_Shin_soo_Choo = 1.0
OF_Willie_Calhoun = 1.0
P_Jacob_deGrom = 1.0
SS_Elvis_Andrus = 1.0
---------------------------------------

Constraints:
3400*1.0 + 2700*1.0 + 2300*1.0 + 2000*1.0 + 2600*1.0 + 3500*1.0 + 3700*1.0 + 11500*1.0 + 3200*1.0 = 34900.0
---------------------------------------

Score:
11.419047619047621*1.0 + 9.407626911745233*1.0 + 7.541052888569079*1.0 + 5.096428462437221*1.0 + 7.5957892166940795*1.0 + 10.937301393539189*1.0 + 11.032142639160156*1.0 + 40.481481481481474*1.0 + 10.493388026213843*1.0 = 114.0042586388879
Variables:

P_Jacob_deGrom = 1.0
---------------------------------------

Constraints:
11500*1.0 = 11500.0
---------------------------------------

Score:
40.481481481481474*1.0 = 40.481481481481474
Variables:

P_Jacob_deGrom = 1.0
---------------------------------------

Constraints:
11500*1.0 = 11500.0
------------------------

Variables:

1B_Danny_Santana = 1.0
2B_Rougned_Odor = 1.0
3B_Logan_Forsythe = 1.0
C_Tim_Federowicz = 1.0
OF_Delino_DeShields = 1.0
OF_Shin_soo_Choo = 1.0
OF_Willie_Calhoun = 1.0
P_Jacob_deGrom = 1.0
SS_Elvis_Andrus = 1.0
---------------------------------------

Constraints:
3400*1.0 + 2700*1.0 + 2300*1.0 + 2000*1.0 + 2600*1.0 + 3500*1.0 + 3700*1.0 + 11500*1.0 + 3200*1.0 = 34900.0
---------------------------------------

Score:
11.419047619047621*1.0 + 9.407626911745233*1.0 + 7.541052888569079*1.0 + 5.096428462437221*1.0 + 7.5957892166940795*1.0 + 10.937301393539189*1.0 + 11.032142639160156*1.0 + 40.481481481481474*1.0 + 10.493388026213843*1.0 = 114.0042586388879
Variables:

1B_Danny_Santana = 1.0
2B_Nick_Solak = 1.0
3B_Starlin_Castro = 1.0
C_Wilkin_Castillo = 1.0
OF_Harold_Ramirez = 1.0
OF_Shin_soo_Choo = 1.0
OF_Willie_Calhoun = 1.0
P_Jacob_deGrom = 1.0
SS_Elvis_Andrus = 1.0
---------------------------------------

Constraints:
3400*1.0 + 2900*1.0 + 2400*1.0 + 2000*1.0 + 2200*1.0 + 3500

# Optimized Stack

In [ ]:
import requests
import pandas as pd
import os
import datetime
import time 
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import glob
import pandas as pd
import urllib, json
import re
from itertools import permutations
from pulp import *

driver = webdriver.Chrome(executable_path=r"C:\\Users\\Matt\\Downloads\\chromedriver.exe")

url = "https://www.fanduel.com/contests/mlb/96"
user = "Matthewdorris7@gmail.com"
password_name = "Gonzaga@1"

driver.get(url)
wait = WebDriverWait(driver,100)

username = wait.until(EC.presence_of_element_located((By.XPATH,"""//*[@id="forms.login.email"]""")))
username.send_keys(user)

password = wait.until(EC.presence_of_element_located((By.XPATH,"""//*[@id="forms.login.password"]""")))
password.send_keys(password_name)

logInBtn = wait.until(EC.element_to_be_clickable((By.XPATH,"""//*[@id="forms.login.submit"]""")))
logInBtn.click()

#logInBtn = wait.until(EC.element_to_be_clickable((By.XPATH,"""/html/body/div[8]/div[2]/i""")))
#logInBtn.click()

sortBtn = wait.until(EC.element_to_be_clickable((By.XPATH,"""//*[@id="root"]/div[1]/main/div/main/div[1]/div[4]/button/span[2]""")))
sortBtn.click()

feeBtn = wait.until(EC.element_to_be_clickable((By.XPATH,"""//*[@id="root"]/div[1]/main/div/main/div[1]/div[4]/div/section/div/div[2]/span""")))
feeBtn.click()

contestBtn = wait.until(EC.element_to_be_clickable((By.XPATH,"""//*[@id="contestList"]/div/div[1]/div/div/div[1]/div/a""")))
contestBtn.click()

playersBtn = wait.until(EC.element_to_be_clickable((By.XPATH,"""//*[@id="global-view"]/div/section/div[2]/roster-editor/div/div[1]/section/div[1]/div/a[2]""")))
playersBtn.click()

list_of_files = glob.glob(r'C:\Users\Matt\Downloads\*.csv') # * means all if need specific format then *.csv
fileName = max(list_of_files, key=os.path.getctime)



In [ ]:
f = open('lineups.csv','w')

def stack_teams(team_1, team_2):

    data = pd.read_csv(fileName)
    data = data[data['Injury Indicator'].isnull()]
    indexNames = data[data['Probable Pitcher']  != 'Yes'].index & data[data['Position']  == 'P'].index
    data.drop(indexNames , inplace=True)
    data = data[(data['Batting Order']  != 0)]
    
    pitchers = data[data['Probable Pitcher']  == 'Yes']
    
    team1 = data[data['Team']  == team_1] 
    team2 = data[data['Team']  == team_2] 
    data= pd.concat([team1, team2])
    data= pd.concat([pitchers, data])
    data.drop(data[data['Position'] == 'P'].index & data[data['Team'] == team_1].index, inplace=True)
    data.drop(data[data['Position'] == 'P'].index & data[data['Team'] == team_2].index, inplace=True)

    current = data[['Position', 'Nickname', 'Salary' , 'FPPG']]

    #current = current.replace('C','C/1B')
    #current = current.replace('1B','C/1B')
    # Add utility position
    #util = current[current.Position.isin(["C/1B","2B","3B", "SS", "OF"])].copy()
    #util.Position = "UTIL"
    #current = pd.concat([current, util])

    availables = current[["Position", "Nickname", "Salary","FPPG"]].groupby(["Position", "Nickname", "Salary", "FPPG"]).agg("count")
    availables = availables.reset_index()
    salaries = {}
    points = {}
    for pos in availables.Position.unique():
        available_pos = availables[availables.Position == pos]
        salary = list(available_pos[["Nickname","Salary"]].set_index("Nickname").to_dict().values())[0]
        point = list(available_pos[["Nickname","FPPG"]].set_index("Nickname").to_dict().values())[0]
        salaries[pos] = salary
        points[pos] = point
    pos_num_available = {
        "P": 1,
        "C": 1,
        "1B": 1,
        "2B": 1,
        "3B": 1, 
        "SS": 1,
        "OF": 3, 
        #"UTIL": 1

    }
    SALARY_CAP = 35000
    _vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}
    prob = LpProblem("Fantasy", LpMaximize)
    rewards = []
    costs = []
    position_constraints = []
    # Setting up the reward
    for k, v in _vars.items():
        costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
        rewards += lpSum([points[k][i] * _vars[k][i] for i in v])
        prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]

    prob += lpSum(rewards)
    prob += lpSum(costs) <= SALARY_CAP
    prob.solve()

    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)

    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
            f.write(str(v.name))
            f.write(",")
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))
    
    f.write(str(eval(score)))
    f.write('\n')


In [96]:
%%time
from fuzzywuzzy import process
str2Match = "Tony Rendon"
strOptions = ["Mike Trout","Anthony Rendon","Juan Soto","Bryce Harper"]
highest = process.extractOne(str2Match,strOptions)
print(highest[0])

Anthony Rendon
Wall time: 2 ms
